In [ ]:
import pandas
import pickle
import datetime
import time
import pytz
import numpy as np
import statistics
import mplfinance as mpf
import matplotlib.ticker as mticker
import io
from PIL import Image
from pathlib import Path


In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)


# GENERATE CHARTS

In [ ]:
def generate_intraday_chart(trade_series, bars_file_path=f'../downloads/bars/SPY_2020-06-14_to_2023-02-16.pickle'):
    bars = pandas.read_pickle(bars_file_path)

    chart_start_hour = 9
    chart_start_minute = 30
    chart_end_hour = 16
    chart_end_minute = 0

    chart_start_timestamp = trade_series['entry_time'].astimezone('US/Eastern').replace(hour=chart_start_hour, minute=chart_start_minute, second=0, microsecond=0, nanosecond=0).astimezone(pytz.utc)
    chart_end_timestamp = trade_series['entry_time'].astimezone('US/Eastern').replace(hour=chart_end_hour, minute=chart_end_minute, second=0, microsecond=0, nanosecond=0).astimezone(pytz.utc)

    trade_start_timestamp = trade_series['entry_time'].replace(second=0, microsecond=0, nanosecond=0)
    
    trade_start_timestamp_minus_6 = x if (x := trade_start_timestamp - pandas.Timedelta(6, "m")) >= chart_start_timestamp else chart_start_timestamp
    trade_start_timestamp_plus_6 = x if (x := trade_start_timestamp + pandas.Timedelta(6, "m")) <= chart_end_timestamp else chart_end_timestamp
    
    trade_start_timestamp_minus_1 = x if (x := trade_start_timestamp - pandas.Timedelta(1, "m")) >= chart_start_timestamp else chart_start_timestamp
    trade_start_timestamp_plus_1 = x if (x := trade_start_timestamp + pandas.Timedelta(1, "m")) <= chart_end_timestamp else chart_end_timestamp

    trade_target_points = [(trade_start_timestamp_minus_6, trade_series['target_price']),
                         (trade_start_timestamp_plus_6, trade_series['target_price']),
                        ]
    trade_stop_points = [(trade_start_timestamp_minus_6, trade_series['stop_price']),
                         (trade_start_timestamp_plus_6, trade_series['stop_price']),
                        ]
    trade_entry_points = [(trade_start_timestamp_minus_1, trade_series['entry_price']),
                         (trade_start_timestamp_plus_1, trade_series['entry_price']),
                        ]
    trade_exit_points = [(trade_start_timestamp_minus_1, trade_series['exit_price']),
                         (trade_start_timestamp_plus_1, trade_series['exit_price']),
                        ]

    level = trade_series['entry_order_price'] + (trade_series['parameter_buffer'] * (-1 if trade_series['direction'] == 'long' else 1))
    trade_level_points = [(chart_start_timestamp, level),
                         (trade_start_timestamp_plus_6, level),
                        ]
    trade_entry_order_points = [(chart_start_timestamp, trade_series['entry_order_price']),
                             (trade_start_timestamp_plus_6, trade_series['entry_order_price']),
                            ]

    exit_color = 'green' if trade_series['r'] > 0 else 'red'

    title = f'{trade_series["r"]}R {trade_series["direction"]} {trade_start_timestamp.strftime("%Y-%m-%d")}'

    downselected_data = bars.loc[chart_start_timestamp:chart_end_timestamp]

    seq_of_points=[
                    trade_level_points,
                    trade_entry_order_points,
                    trade_target_points,
                    trade_stop_points,
                    trade_entry_points,
                    trade_exit_points
                  ]

    mc = mpf.make_marketcolors(up='#26A69A',down='#EF5350',inherit=True)
    s  = mpf.make_mpf_style(base_mpf_style='yahoo', gridstyle='', marketcolors=mc, facecolor='#ffffff')

    plot, axlist = mpf.plot(downselected_data, type='candle', style=s, volume=True, show_nontrading=False, returnfig=True, ylabel='', scale_width_adjustment=dict(candle=1.5, volume=0.75), alines=dict(alines=seq_of_points, colors=['gray', 'gray', 'green', 'red', 'black', exit_color], linestyle=['solid', 'dotted', 'dotted', 'dotted', 'solid', 'solid']), mav=(), panel_ratios=(7, 1), figsize=(22, 16))
    plot.suptitle(title, fontsize=21, x=.42, y=.901)

    axlist[0].set_yscale('linear')
    formatter = mticker.ScalarFormatter(useMathText=False)
    formatter.set_scientific(False)
    axlist[0].yaxis.set_major_formatter(formatter)
    axlist[0].yaxis.set_minor_formatter(formatter)

    axlist[2].yaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))
    axlist[2].yaxis.set_label_text("")

    img_buf = io.BytesIO()
    plot.savefig(img_buf, bbox_inches="tight")
    image = Image.open(img_buf)
    return image


def generate_daily_chart(trade_series, num_daily_bars=25, bars_file_path=f'../downloads/bars/SPY_DAILY_2020-06-14_to_2023-02-16.pickle'):
    bars = pandas.read_pickle(bars_file_path)

    chart_start_timestamp = (trade_series['entry_time'].replace(hour=0, minute=0, second=0, microsecond=0, nanosecond=0) - pandas.Timedelta(num_daily_bars, "d"))
    chart_end_timestamp = trade_series['entry_time'].replace(hour=0, minute=0, second=0, microsecond=0, nanosecond=0)

    downselected_data = bars.loc[chart_start_timestamp:chart_end_timestamp]

    level = trade_series['entry_order_price'] + (trade_series['parameter_buffer'] * (-1 if trade_series['direction'] == 'long' else 1))
    trade_level_points = [(downselected_data.iloc[0].name, level),
                          (downselected_data.iloc[-1].name, level),
                         ]
    trade_entry_order_points = [(downselected_data.iloc[0].name, trade_series['entry_order_price']),
                                (downselected_data.iloc[-1].name, trade_series['entry_order_price']),
                               ]

    title = f'level: {level:0.2f}  order: {trade_series["entry_order_price"]:0.2f}'


    seq_of_points=[
                    trade_level_points,
                    trade_entry_order_points,
                  ]

    mc = mpf.make_marketcolors(up='#26A69A',down='#EF5350',inherit=True)
    s  = mpf.make_mpf_style(base_mpf_style='yahoo', gridstyle='', marketcolors=mc, facecolor='#ffffff')

    plot, axlist = mpf.plot(downselected_data, type='candle', style=s, volume=True, show_nontrading=False, returnfig=True, ylabel='', scale_width_adjustment=dict(candle=1.0, volume=0.75), alines=dict(alines=seq_of_points, colors=['gray', 'gray'], linestyle=['solid', 'dotted']), mav=(9,20), panel_ratios=(7, 1), figsize=(6, 16))
    plot.suptitle(title, fontsize=10, x=.42, y=.901)

    axlist[0].set_yscale('linear')
    formatter = mticker.ScalarFormatter(useMathText=False)
    formatter.set_scientific(False)
    axlist[0].yaxis.set_major_formatter(formatter)
    axlist[0].yaxis.set_minor_formatter(formatter)

    axlist[2].yaxis.set_major_formatter(mticker.FormatStrFormatter('%d'))
    axlist[2].yaxis.set_label_text("")
    axlist[2].xaxis.set_tick_params(labelsize='small')


    img_buf = io.BytesIO()
    
    plot.savefig(img_buf, bbox_inches="tight")
    del plot
    
    image = Image.open(img_buf)
    return image


def generate_chart(trade_series):
    daily_image = generate_daily_chart(trade_series)
    intraday_image = generate_intraday_chart(trade_series)
    
    images = [daily_image, intraday_image]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_image = Image.new('RGB', (total_width, max_height), color="white")

    x_offset = 0
    for im in images:
        new_image.paste(im, (x_offset,0))
        x_offset += im.size[0]

#     new_image.save('test.jpg')
    return new_image


def generate_all_charts(trades_df, output_dir='output'):
    winning_dir = f"{output_dir}/winning"
    loosing_dir = f"{output_dir}/loosing"
    Path(winning_dir).mkdir(parents=True, exist_ok=True)
    Path(loosing_dir).mkdir(parents=True, exist_ok=True)

    print(f'Processing {trades_df.shape[0]} ttl trades.')
    count = 1
    for index, row in trades_df.iterrows():
        print(f'processing {count:3}')
        count += 1
        is_positive = row['r'] > 0
        image = generate_chart(row)
        
        directory =  winning_dir if is_positive else loosing_dir
        file_name = count
        file = f'{directory}/{file_name}.jpg'
        image.save(file)


# GENERATE RESULTS

In [ ]:
# GET AND DOWN_SELECT TRADES
df = pandas.read_pickle("../output/trades_with_derivatives.pickle")

direction = 'long'
min_distance = 0.7
max_rvol = 1.4
buffer = 0.2
stop_a = 0.5
stop_b = 0.0
target_a = 0.2
target_b = 0.0


trades = df[
             (df['direction'] == direction) &
             (df['distance'] >= min_distance) &
             (df['rvol_5_SMA_15min'] <= max_rvol) &
             (df['parameter_buffer'] == buffer) &
             (df['parameter_stop_a'] == stop_a) &
             (df['parameter_stop_b'] == stop_b) &
             (df['parameter_target_a'] == target_a) &
             (df['parameter_target_b'] == target_b)
           ]

generate_all_charts(trades)